In [1]:
import sys
sys.path.append('/home/agilex/dexhand/repo/piper-ik-test/venv/lib/python3.10/site-packages/cmeel.prefix/lib/python3.10/site-packages')

# print(sys.path)

import pinocchio as pin
import numpy as np
import os

import pinocchio.visualize

# Set up matplotlib for notebook display
%matplotlib inline

# set ROS_PACKAGE_PATH
os.environ["ROS_PACKAGE_PATH"] = "/home/agilex/dexhand/dexhand_ws/install/piper_description/share/"

print(os.environ["ROS_PACKAGE_PATH"])

/home/agilex/dexhand/dexhand_ws/install/piper_description/share/


In [29]:
from pathlib import Path
from sys import argv

import pinocchio as pin
from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.visualize import MeshcatVisualizer

# pinocchio_model_dir = Path(__file__).parent / "external" / "example-robot-data"/ "robots"
model_dir = Path(os.environ["ROS_PACKAGE_PATH"]) / "piper_description"
mesh_dir = model_dir / "meshes"
urdf_file = model_dir / "urdf" / "piper_no_gripper_description.urdf"
mjcf_file = model_dir / "mujoco_model" / "piper_no_gripper_description.xml"

print (model_dir)
print(mesh_dir)
print(urdf_file)
print(mjcf_file)


root = pin.JointModelFreeFlyer()
# robot = RobotWrapper.BuildFromURDF(urdf_file, root_joint=root, verbose=True)
robot = RobotWrapper.BuildFromMJCF(mjcf_file, root_joint=root, verbose=True)

def quaternion_from_euler(euler_angles):
    """
    Convert Euler angles to quaternion.
    
    Args:   
        euler_angles: tuple of 3 floats, representing the Euler angles (roll, pitch, yaw)
    Returns:
        numpy array of 4 floats, representing the quaternion (x, y, z, w)
    """
    from scipy.spatial.transform import Rotation as R
    
    # Convert Euler angles (roll, pitch, yaw) to quaternion
    r = R.from_euler('xyz', euler_angles)
    quat = r.as_quat()  # Returns (x, y, z, w)
    return quat

q = pin.Quaternion(quaternion_from_euler([0, -1.57, -1.57]))

robot.model.addFrame(pin.Frame(
    "ee_frame",
    robot.model.getJointId("joint6"),
    pin.SE3(q, np.zeros(3)),
    pin.FrameType.OP_FRAME
))
# print all frames
robot.data.oMf[robot.model.getFrameId("ee_frame") - 1].translation = np.zeros(3)


init_data = np.zeros(robot.model.nq)
history_data = np.zeros(robot.model.nq)

# print all frame ids
print(robot.model.frames)

viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel("pinocchio/piper")
viz.displayFrames(True, frame_ids=[robot.model.getFrameId("ee_frame")], axis_length=0.2, axis_width=2)
q = pin.neutral(robot.model)
viz.display(q)


/home/agilex/dexhand/dexhand_ws/install/piper_description/share/piper_description
/home/agilex/dexhand/dexhand_ws/install/piper_description/share/piper_description/meshes
/home/agilex/dexhand/dexhand_ws/install/piper_description/share/piper_description/urdf/piper_no_gripper_description.urdf
/home/agilex/dexhand/dexhand_ws/install/piper_description/share/piper_description/mujoco_model/piper_no_gripper_description.xml
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/


In [30]:
# Enable the display of end effector target frames with short axis lengths and greater width.
frame_viz_names = ['ee_target']
FRAME_AXIS_POSITIONS = (
    np.array([[0, 0, 0], [1, 0, 0],
                [0, 0, 0], [0, 1, 0],
                [0, 0, 0], [0, 0, 1]]).astype(np.float32).T
)
FRAME_AXIS_COLORS = (
    np.array([[1, 0, 0], [1, 0.6, 0],
                [0, 1, 0], [0.6, 1, 0],
                [0, 0, 1], [0, 0.6, 1]]).astype(np.float32).T
)
axis_length = 0.1
axis_width = 10

import meshcat.geometry as mg

for frame_viz_name in frame_viz_names:
    viz.viewer[frame_viz_name].set_object(
        mg.LineSegments(
            mg.PointsGeometry(
                position=axis_length * FRAME_AXIS_POSITIONS,
                color=FRAME_AXIS_COLORS,
            ),
            mg.LineBasicMaterial(
                linewidth=axis_width,
                vertexColors=True,
            ),
        )
    )

In [38]:
from pinocchio import casadi as cpin
import casadi
import meshcat.geometry as mg

class ArmIK:
    def __init__(self):
        np.set_printoptions(precision=5, suppress=True, linewidth=200)

        self.robot = pin.RobotWrapper.BuildFromMJCF(mjcf_file)

        self.mixed_jointsToLockIDs = []

        self.reduced_robot = self.robot.buildReducedRobot(
            list_of_joints_to_lock=self.mixed_jointsToLockIDs,
            reference_configuration=np.array([0] * self.robot.model.nq),
        )

        q = quaternion_from_euler((0, -1.57, -1.57))
        self.reduced_robot.model.addFrame(
            pin.Frame('ee',
                      self.reduced_robot.model.getJointId('joint6'),
                      pin.SE3(
                          # pin.Quaternion(1, 0, 0, 0),
                          pin.Quaternion(q[3], q[0], q[1], q[2]),
                          np.array([0.0, 0.0, 0.0]),
                      ),
                      pin.FrameType.OP_FRAME)
        )

        # self.geom_model = pin.buildGeomFromUrdf(self.robot.model, urdf_path, pin.GeometryType.COLLISION)
        # for i in range(4, 9):
        #     for j in range(0, 3):
        #         self.geom_model.addCollisionPair(pin.CollisionPair(i, j))
        # self.geometry_data = pin.GeometryData(self.geom_model)

        self.init_data = np.zeros(self.reduced_robot.model.nq)
        self.history_data = np.zeros(self.reduced_robot.model.nq)

        # # Initialize the Meshcat visualizer  for visualization
        self.vis = MeshcatVisualizer(self.reduced_robot.model, self.reduced_robot.collision_model, self.reduced_robot.visual_model)
        self.vis.initViewer(open=True)
        self.vis.loadViewerModel("pinocchio")
        self.vis.displayFrames(True, frame_ids=[113, 114], axis_length=0.15, axis_width=5)
        self.vis.display(pin.neutral(self.reduced_robot.model))

        # Enable the display of end effector target frames with short axis lengths and greater width.
        frame_viz_names = ['ee_target']
        FRAME_AXIS_POSITIONS = (
            np.array([[0, 0, 0], [1, 0, 0],
                      [0, 0, 0], [0, 1, 0],
                      [0, 0, 0], [0, 0, 1]]).astype(np.float32).T
        )
        FRAME_AXIS_COLORS = (
            np.array([[1, 0, 0], [1, 0.6, 0],
                      [0, 1, 0], [0.6, 1, 0],
                      [0, 0, 1], [0, 0.6, 1]]).astype(np.float32).T
        )
        axis_length = 0.1
        axis_width = 10
        for frame_viz_name in frame_viz_names:
            self.vis.viewer[frame_viz_name].set_object(
                mg.LineSegments(
                    mg.PointsGeometry(
                        position=axis_length * FRAME_AXIS_POSITIONS,
                        color=FRAME_AXIS_COLORS,
                    ),
                    mg.LineBasicMaterial(
                        linewidth=axis_width,
                        vertexColors=True,
                    ),
                )
            )

        # Creating Casadi models and data for symbolic computing
        self.cmodel = cpin.Model(self.reduced_robot.model)
        self.cdata = self.cmodel.createData()

        # Creating symbolic variables
        self.cq = casadi.SX.sym("q", self.reduced_robot.model.nq, 1)
        self.cTf = casadi.SX.sym("tf", 4, 4)
        cpin.framesForwardKinematics(self.cmodel, self.cdata, self.cq)

        # # Get the hand joint ID and define the error function
        self.gripper_id = self.reduced_robot.model.getFrameId("ee")
        self.error = casadi.Function(
            "error",
            [self.cq, self.cTf],
            [
                casadi.vertcat(
                    cpin.log6(
                        self.cdata.oMf[self.gripper_id].inverse() * cpin.SE3(self.cTf)
                    ).vector,
                )
            ],
        )

        # Defining the optimization problem
        self.opti = casadi.Opti()
        self.var_q = self.opti.variable(self.reduced_robot.model.nq)
        # self.var_q_last = self.opti.parameter(self.reduced_robot.model.nq)   # for smooth
        self.param_tf = self.opti.parameter(4, 4)
        self.totalcost = casadi.sumsqr(self.error(self.var_q, self.param_tf))
        self.regularization = casadi.sumsqr(self.var_q)
        # self.smooth_cost = casadi.sumsqr(self.var_q - self.var_q_last) # for smooth

        # Setting optimization constraints and goals
        self.opti.subject_to(self.opti.bounded(
            self.reduced_robot.model.lowerPositionLimit,
            self.var_q,
            self.reduced_robot.model.upperPositionLimit)
        )
        # print("self.reduced_robot.model.lowerPositionLimit:", self.reduced_robot.model.lowerPositionLimit)
        # print("self.reduced_robot.model.upperPositionLimit:", self.reduced_robot.model.upperPositionLimit)
        self.opti.minimize(20 * self.totalcost + 0.01 * self.regularization)
        # self.opti.minimize(20 * self.totalcost + 0.01 * self.regularization + 0.1 * self.smooth_cost) # for smooth

        opts = {
            'ipopt': {
                'print_level': 0,
                'max_iter': 50,
                'tol': 1e-4
            },
            'print_time': False
        }
        self.opti.solver("ipopt", opts)

    def ik_func(self, target_pose, gripper=0, motorstate=None, motorV=None):
        gripper = np.array([gripper/2.0, -gripper/2.0])
        if motorstate is not None:
            self.init_data = motorstate
        self.opti.set_initial(self.var_q, self.init_data)

        self.vis.viewer['ee_target'].set_transform(target_pose)     # for visualization

        self.opti.set_value(self.param_tf, target_pose)
        # self.opti.set_value(self.var_q_last, self.init_data) # for smooth

        try:
            # sol = self.opti.solve()
            sol = self.opti.solve_limited()
            sol_q = self.opti.value(self.var_q)

            if self.init_data is not None:
                max_diff = max(abs(self.history_data - sol_q))
                # print("max_diff:", max_diff)
                self.init_data = sol_q
                if max_diff > 30.0/180.0*3.1415:
                    # print("Excessive changes in joint angle:", max_diff)
                    self.init_data = np.zeros(self.reduced_robot.model.nq)
            else:
                self.init_data = sol_q
            self.history_data = sol_q

            self.vis.display(sol_q)  # for visualization

            if motorV is not None:
                v = motorV * 0.0
            else:
                v = (sol_q - self.init_data) * 0.0

            tau_ff = pin.rnea(self.reduced_robot.model, self.reduced_robot.data, sol_q, v,
                              np.zeros(self.reduced_robot.model.nv))

            is_collision = self.check_self_collision(sol_q, gripper)

            return sol_q, tau_ff, not is_collision

        except Exception as e:
            print(f"ERROR in convergence, plotting debug info.{e}")
            # sol_q = self.opti.debug.value(self.var_q)   # return original value
            return sol_q, '', False

    def check_self_collision(self, q, gripper=np.array([0, 0])):
        pin.forwardKinematics(self.robot.model, self.robot.data, np.concatenate([q, gripper], axis=0))
        pin.updateGeometryPlacements(self.robot.model, self.robot.data, self.geom_model, self.geometry_data)
        collision = pin.computeCollisions(self.geom_model, self.geometry_data, False)
        # print("collision:", collision)
        return collision

    def get_ik_solution(self, x,y,z,roll,pitch,yaw):
        
        q = quaternion_from_euler((roll, pitch, yaw))
        target = pin.SE3(
            pin.Quaternion(q[3], q[0], q[1], q[2]),
            np.array([x, y, z]),
        )
        sol_q, tau_ff, get_result = self.ik_func(target.homogeneous,0)
        # print("result:", sol_q)
        
        if get_result :
            print("send to hardware.")
            # piper_control.joint_control_piper(sol_q[0],sol_q[1],sol_q[2],sol_q[3],sol_q[4],sol_q[5],0)
        else :
            print("collision!!!")

ImportError: cannot import name 'casadi' from 'pinocchio' (/home/agilex/dexhand/repo/piper-ik-test/venv/lib/python3.10/site-packages/cmeel.prefix/lib/python3.10/site-packages/pinocchio/__init__.py)

In [37]:
# create a Arm-IK and visualize it
ik = ArmIK()

p = [1, 1, 1]
roll = 0
pitch = 0
yaw = 0

ik.get_ik_solution(p[0], p[1], p[2], roll, pitch, yaw)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/


ArgumentError: Python argument types in
    None.framesForwardKinematics(Model, Data, SX)
did not match C++ signature:
    framesForwardKinematics(pinocchio::ModelTpl<double, 0, pinocchio::JointCollectionDefaultTpl> model, pinocchio::DataTpl<double, 0, pinocchio::JointCollectionDefaultTpl> {lvalue} data, Eigen::MatrixBase<Eigen::Matrix<double, -1, 1, 0, -1, 1> > q)